# Fichier principal

Dans ce notebook, nous utilisons le jeu de donnée des accidents de la route. <br>
Tout d'abord en ananlysant les données brutes.<br>
Puis en les transformant en un modèle à l'aide d'une **forêt aléatoire** utilisant un seuil de prédiction modifié.<br>
Pour finir, nous analysons ce modèle.

On souhaite prédire si un véhicule donné est impliqué dans un accident **mortel** ou non.

In [23]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from utils import *

import pickle
import dice_ml

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing

from dataset_prepare import load_dataset, pred_thres, test_train_sets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Préparation du dataframe
Les attributs présents dans la base de données ne sont pas tous utiles et certains doivent être transformer avant d'être utilisé.

Pour simplifier le code, les fonction qui uniformisent les types vers ceux souhaités ont été plcées dans **utils.py**

In [24]:
df_2 = load_dataset()
label = 'mortal'
df_2

/home/erwan/Univ/L3/fairness/fairness/dataset_prepare.py:18: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



,Num_Acc,trajet,catr,circ,nbv,prof,plan,surf,infra,situ,...,atm,col,catv,obs,obsm,choc,mortal,pieton,sexe_conducteur,age
0,202200000001,5,4,2,2,1,1,1,0,1,...,1,3,2,0,2,1,0,0,1,14
1,202200000001,5,4,2,2,1,1,1,0,1,...,1,3,2,0,2,1,0,0,1,74
2,202200000002,0,4,2,2,1,1,1,0,1,...,1,3,3,0,2,8,0,0,1,34
3,202200000002,4,4,2,2,1,1,1,0,1,...,1,3,3,0,2,8,0,0,1,52
4,202200000003,0,3,-1,2,1,1,1,5,1,...,1,2,3,0,2,1,0,0,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126656,202200055300,5,3,2,2,1,2,7,0,3,...,9,6,3,1,0,1,1,0,1,27
126657,202200055301,5,3,2,2,1,1,1,0,1,...,1,3,3,0,0,8,0,0,0,20
126659,202200055301,5,3,2,2,1,1,1,0,1,...,1,3,3,0,0,8,0,0,0,69
126660,202200055302,1,3,3,4,1,1,1,0,1,...,1,2,5,0,2,1,0,0,1,30


Tri manuel des valeurs catégorielles/numériques

In [25]:
for column in df_2.columns:
  if df_2[column].isnull().values.any() == True:
    print(column, df_2[column].isnull().values.any()) # afficher s'il y a des valeurs nulles

In [26]:
# valeurs catégorielles
categorical_features = ['trajet', 'catr', 'circ', 'nbv', 'prof',
                        'plan', 'surf', 'vma', 'lum', 'agg', 
                        'int', 'atm', 'col', 'catv', 'obs', 'obsm', 'choc', 'pieton',
                        'sexe_conducteur', 'infra', 'situ']
# valeurs numériques
numerical_features = ['dep', 'mois', 'age']

print("numerical : ", numerical_features)
print("categorical : ", categorical_features)

numerical :  ['dep', 'mois', 'age']
categorical :  ['trajet', 'catr', 'circ', 'nbv', 'prof', 'plan', 'surf', 'vma', 'lum', 'agg', 'int', 'atm', 'col', 'catv', 'obs', 'obsm', 'choc', 'pieton', 'sexe_conducteur', 'infra', 'situ']


## Analyse de données
**Attention** conserver les sorties alourdit grandement le notebook, **clear all outputs** avant d'enregistrer est à privilégier.

### Analyse univariée

In [5]:
val = [len(df_2[df_2.mortal == 1]), len(df_2[df_2.mortal == 0])]
labels = ['Accident mortel', 'Accident non mortel']
px.pie(values=val, names=labels)

In [6]:
val = [len(df_2[df_2.pieton == 1]), len(df_2[df_2.pieton == 0])]
labels = ['Implique piéton', 'N\'implique pas de piéton']
px.pie(values=val, names=labels)

In [ ]:
fig = px.histogram(df_2['catv'].replace({0:"Autre", 1:"Bicyclette", 2:"Cyclomoteur", 3:"Voiture", 4:"Utilitaire", 5:"Moto"}), x="catv")
fig.show()

fig = px.box(df_2, x="age")
fig.show()

fig = px.histogram(df_2, x="age")
fig.show()

fig = px.histogram(df_2['sexe_conducteur'].replace({1:"Homme", 0:"Femme"}), x="sexe_conducteur")
fig.show()

fig = px.histogram(df_2['catr'].replace({1:"Autoroute", 2:"Route nationale", 3:"Route départementale", 4:"Voie communale", 5: "Autre"}) , x="catr")
fig.show()

fig = px.histogram(df_2['col'].replace(
    {-1:"Non renseigné", 
     1:"Deux véhicules - Fontale", 
     2:"Deux véhicules - par l'arrière", 
     3:"Deux véhicules - par le côté", 
     4:"Trois véhicules et plus en chaine", 
     5:"Trois véhicules et plus collisions multiples",
     6:"Autre collision", 
     7: "Pas de collision"}), x="col")
fig.show()

fig = px.histogram(df_2['obs'].replace({0:"Pas d'obstacle", 1:"Obstacle"}), x='obs')
fig.show()

### Analyse bivariée

In [ ]:
features = ['age', 'mois', 'catr', 'mortal', 'sexe_conducteur', 'col']

for exp in features:
    print(exp)
    analyse_bi_quali_quanti("catv", exp, df_2)

In [ ]:
features = ['age', 'mois', 'catr', 'mortal', 'col']

for exp in features:
    print(exp)
    analyse_bi_quali_quanti("vma", exp, df_2)

In [ ]:
features = ['age', 'vma', 'mois', 'catr', 'mortal']

for exp in features:
    print(exp)
    analyse_bi_quali_quanti("col", exp, df_2)

In [ ]:
features = ['age', 'vma', 'mois', 'catr', 'mortal']

interet = "pieton"

for exp in categorical_features:
    if exp==interet:
        continue
    print(exp)
    analyse_bi_quali_quali(interet, exp, df_2, numerical_features)

In [ ]:
interet = "agg"

for exp in categorical_features:
    if exp==interet:
        continue
    print(exp)
    analyse_bi_quali_quali(interet, exp, df_2, numerical_features)

In [ ]:
interet = "sexe_conducteur"

for exp in categorical_features:
    if exp==interet:
        continue
    print(exp)
    analyse_bi_quali_quali(interet, exp, df_2, numerical_features)

## Apprentissage avec un arbre de décision

In [27]:
# Récupération des ensembles de train/test
X_train, X_test, y_train, y_test = test_train_sets(df_2)
df_2 = df_2.drop(columns='Num_Acc')

In [28]:
from utils import custom_RFC

clf = pickle.load(open('models/rfc_model.sav', 'rb'))
preds = clf.predict(X_test)
clf.score(X_train, y_train), clf.score(X_test, y_test)

(0.9963024885859431, 0.8780006576783953)

### Afficher la matrice de confusion

In [29]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
tn, fp, fn, tp

(26016, 2701, 1009, 684)

In [30]:
import plotly.express as px

fig = px.imshow([[tn, fp], [fn, tp]], text_auto=True, labels=dict(y="Truth", x="Pred"),
                x=["False", "True"],
                y=["False", "True"]
               )
fig.show()

### Les métriques "Base rate"

In [31]:
encoders = {cat_col:preprocessing.LabelEncoder() for cat_col in categorical_features}

for cat_col in categorical_features:
  df_2[cat_col] = encoders[cat_col].fit_transform(df_2[cat_col])
  print(cat_col)
  for idx in sorted(df_2[cat_col].unique()):
    print(idx, encoders[cat_col].inverse_transform([idx]))

trajet
0 [0]
1 [1]
2 [2]
3 [3]
4 [4]
5 [5]
catr
0 [1]
1 [2]
2 [3]
3 [4]
4 [5]
circ
0 [-1]
1 [1]
2 [2]
3 [3]
4 [4]
nbv
0 [-1]
1 [0]
2 [1]
3 [2]
4 [3]
5 [4]
6 [5]
7 [6]
8 [7]
9 [8]
10 [9]
11 [10]
12 [11]
13 [12]
prof
0 [-1]
1 [1]
2 [2]
3 [3]
4 [4]
plan
0 [-1]
1 [1]
2 [2]
3 [3]
4 [4]
surf
0 [0]
1 [1]
2 [2]
3 [3]
4 [4]
5 [5]
6 [6]
7 [7]
8 [8]
vma
0 [1]
1 [2]
2 [3]
3 [4]
lum
0 [-1]
1 [1]
2 [2]
3 [3]
4 [4]
5 [5]
agg
0 [1]
1 [2]
int
0 [-1]
1 [1]
2 [2]
3 [3]
4 [4]
5 [5]
6 [6]
7 [7]
8 [8]
9 [9]
atm
0 [-1]
1 [1]
2 [2]
3 [3]
4 [4]
5 [5]
6 [6]
7 [7]
8 [8]
9 [9]
col
0 [-1]
1 [1]
2 [2]
3 [3]
4 [4]
5 [5]
6 [6]
7 [7]
catv
0 [0]
1 [1]
2 [2]
3 [3]
4 [4]
5 [5]
obs
0 [0]
1 [1]
obsm
0 [-1]
1 [0]
2 [1]
3 [2]
4 [4]
5 [5]
6 [6]
7 [9]
choc
0 [-1]
1 [0]
2 [1]
3 [2]
4 [3]
5 [4]
6 [5]
7 [6]
8 [7]
9 [8]
10 [9]
pieton
0 [0]
1 [1]
sexe_conducteur
0 [0]
1 [1]
infra
0 [-1]
1 [0]
2 [1]
3 [2]
4 [3]
5 [4]
6 [5]
7 [6]
8 [7]
9 [8]
10 [9]
situ
0 [-1]
1 [1]
2 [2]
3 [3]
4 [4]
5 [5]
6 [6]
7 [8]


In [32]:
def compute_stat(preds, sensitive, value=None, not_value=None):
    if value is not None:
      preds_sel = preds[sensitive == value]
    elif not_value is not None:
      preds_sel = preds[sensitive != not_value]
    else:
      print("one of 'value' and 'not value' must be set")
      return None, None, None, None
    card = len(preds_sel)
    card_pos = len(preds_sel[preds_sel == 1])
    card_neg = len(preds_sel[preds_sel == 0])
    return card, card_pos, card_neg, preds_sel


def compute_baserate(preds, sensitive=X_test["sexe_conducteur"], value=0):

    n_F, n_F_pos, n_F_neg, preds_F = compute_stat(preds=preds, sensitive=sensitive, value=value)
    n_M, n_M_pos, n_M_neg, preds_M = compute_stat(preds=preds, sensitive=sensitive, not_value=value)

    DI = n_F_pos/n_F*n_M/n_M_pos
    p_DI = np.min([DI, 1/DI])
    DP = n_F_pos/n_F - n_M_pos/n_M

    ret = {}
    ret["disparate_impact"] = DI
    ret["P_rule_disparate_impact"] = p_DI
    ret["demography_parity"] = DP

    return ret

In [33]:
preds = clf.predict(X_test)
for att,unpriv in zip(['sexe_conducteur', 'pieton'],[1, 1]):
  value = encoders[att].transform([unpriv])[0]
  base_rate_pred = compute_baserate(preds, sensitive=X_test[att], value=value)
  base_rate_label = compute_baserate(y_test, sensitive=X_test[att], value=value)
  print(att, unpriv)
  for k,v in base_rate_pred.items():
    print(k, v, base_rate_label[k])

sexe_conducteur 1
disparate_impact 1.9440503811733512 1.5733869178937117
P_rule_disparate_impact 0.5143899611266453 0.6355715740529335
demography_parity 0.061968798464117195 0.022439373005842936
pieton 1
disparate_impact 0.7453968803689579 1.0800450805571133
P_rule_disparate_impact 0.7453968803689579 0.9258872782274753
demography_parity -0.02904961032525566 0.004422364056385214


### Le taux d'erreur dans pour les attributs sensibles

P(Y^=1| (Y=1, Z=0)) <br> P(Y^=1| (Y=0, Z=0)) <br> P(Y^=1| (Y=1, Z=1)) <br> P(Y^=1| (Y=0, Z=1))

In [34]:
def sensible_error_rate (X, desired_output, group, target, label):
   cardGlob1 = len(X[(X[label]==1) & (X[target] == group)])
   cardGlob2 = len(X[(X[label]==0) & (X[target] == group)])
   cardGlob3 = len(X[(X[label]==1) & (X[target] == np.absolute(group-1))])
   cardGlob4 = len(X[(X[label]==0) & (X[target] == np.absolute(group-1))])
   cardPred1 = len(X[(X['pred']==desired_output) & (X[label]==1) & (X[target] == group)])
   cardPred2 = len(X[(X['pred']==desired_output) & (X[label]==0) & (X[target] == group)])
   cardPred3 = len(X[(X['pred']==desired_output) & (X[label]==1) & (X[target] == np.absolute(group-1))])
   cardPred4 = len(X[(X['pred']==desired_output) & (X[label]==0) & (X[target] == np.absolute(group-1))])

   return cardPred1/cardGlob1, cardPred2/cardGlob2, cardPred3/cardGlob3, cardPred4/cardGlob4

In [35]:
target = 'sexe_conducteur'
X_error_rate_1 = pd.DataFrame()
X_error_rate_1[target] = X_test[target]
X_error_rate_1[label] = y_test
X_error_rate_1['pred'] = preds
result = sensible_error_rate (X_error_rate_1, 1, 0, target, label)
print("Les femmes conductrices impliquées dans un accident mortel : {0} \nLes femmes conductrices impliquées dans un accident non mortel : {1} \nLes hommes conducteurs impliquées dans un accident mortel : {2} \nLes hommes conducteurs impliquées dans un accident non mortel : {3} \n".format(result[0], result[1], result[2], result[3]))

Les femmes conductrices impliquées dans un accident mortel : 0.30670926517571884 
Les femmes conductrices impliquées dans un accident non mortel : 0.05582303188028627 
Les hommes conducteurs impliquées dans un accident mortel : 0.4260869565217391 
Les hommes conducteurs impliquées dans un accident non mortel : 0.10802586534804108 



In [36]:
target = 'pieton'
X_error_rate_1 = pd.DataFrame()
X_error_rate_1[target] = X_test[target]
X_error_rate_1[label] = y_test
X_error_rate_1['pred'] = preds
print(sensible_error_rate (X_error_rate_1, 1, 0, target, label))

(0.40026333113890716, 0.0973628488931665, 0.4367816091954023, 0.06272793581327499)


## Audit du modèle

In [37]:
import pickle

data_test = X_test.copy(deep=True)
data_test["Y"] = y_test

data_test.to_csv("test_data.csv",
          index=False)

data_train = X_train.copy(deep=True)
data_train["Y"] = y_train

data_train.to_csv("train_data.csv",
          index=False)

with open( 'clf.pickle', 'wb' ) as f:
    pickle.dump(clf, f )

### Trouver des contrefactuels avec Dice

In [15]:
train_dataset = X_train.copy(deep=True)
train_dataset[label] = y_train

d = dice_ml.Data(dataframe=train_dataset, continuous_features=numerical_features, outcome_name=label)

m = dice_ml.Model(model=clf, backend="sklearn")

exp = dice_ml.Dice(d, m, method="random")

In [21]:
# valeurs prédites vraies
p = []
predict = clf.predict(X_test)
for i in range(0, len(predict)):
    if predict[i] == 1.:
        p.append(i)

In [28]:
for i in range(0, 5):
    r = random.randint(0, 100)
    e1 = exp.generate_counterfactuals(X_test[p[i]:p[i]+1], total_CFs=10, desired_class="opposite")
    e1.visualize_as_dataframe(show_only_changes=True)
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    imp = exp.local_feature_importance(X_test[p[i]:p[i]+1], cf_examples_list=e1.cf_examples_list)
    print(imp.local_importance)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.60it/s]

Query instance (original outcome : 1)


,trajet,catr,circ,nbv,prof,plan,surf,infra,situ,vma,mois,lum,dep,agg,int,atm,col,catv,obs,obsm,choc,pieton,sexe_conducteur,age,mortal
0,5,3,2,2,1,1,1,0,1,3,10,1,51,1,1,1,1,3,0,2,1,0,1,22.0,1



Diverse Counterfactual set (new outcome: 0)


...

Query instance (original outcome : 1)


,trajet,catr,circ,nbv,prof,plan,surf,infra,situ,vma,mois,lum,dep,agg,int,atm,col,catv,obs,obsm,choc,pieton,sexe_conducteur,age,mortal
0,5,3,2,2,1,1,1,0,1,3,10,3,45,1,1,1,2,4,0,2,3,0,1,32.0,1



Diverse Counterfactual set (new outcome: 0)


,trajet,catr,circ,nbv,prof,plan,surf,infra,situ,vma,mois,lum,dep,agg,int,atm,col,catv,obs,obsm,choc,pieton,sexe_conducteur,age,mortal
0,-,-,-,-,-,-,-,-,-,-,-,-,556.0,-,-,-,-,-,-,-,-,-,-,19.6,0.0
1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0,10.3,0.0
2,-,-,-,-,-,-,-,7.0,-,-,-,-,-,-,-,-,-,-,-,-,-,1.0,-,-,0.0
3,-,-,-,-,-,-,-,-,-,-,2.0,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0
4,-,-,-,-,-,-,-,-,-,-,-,-,687.0,-,-,4.0,-,-,-,-,-,-,-,-,0.0
5,-,-,-,-,-,-,-,-,-,-,2.0,-,-,-,-,-,-,-,-,-,-,-,0.0,-,0.0
6,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,4.0,-,-,-,-,-,-,-,0.0
7,-,-,1.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0
8,-,-,-,-,-,-,-,-,-,-,-,-1.0,-,-,-,2.0,-,-,-,-,-,-,-,-,0.0
9,4.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0


[{'atm': 0.2, 'sexe_conducteur': 0.2, 'dep': 0.2, 'mois': 0.2, 'age': 0.2, 'trajet': 0.1, 'circ': 0.1, 'infra': 0.1, 'lum': 0.1, 'col': 0.1, 'pieton': 0.1, 'catr': 0.0, 'nbv': 0.0, 'prof': 0.0, 'plan': 0.0, 'surf': 0.0, 'situ': 0.0, 'vma': 0.0, 'agg': 0.0, 'int': 0.0, 'catv': 0.0, 'obs': 0.0, 'obsm': 0.0, 'choc': 0.0}]


### BlackBoxAuditing

In [38]:
from BlackBoxAuditing.data import load_from_file
from BlackBoxAuditing.model_factories.AbstractModelFactory import AbstractModelFactory
from BlackBoxAuditing.model_factories.AbstractModelVisitor import AbstractModelVisitor

import BlackBoxAuditing as BBA


(_, train_BBA, _, _, _, _) = load_from_file("train_data.csv",
                      correct_types = np.repeat([int], [len(data_test.columns)]),
                                response_header = 'Y',
                               train_percentage = 1.0)
(headers, _, test_BBA, response_header, features_to_ignore, correct_types) = load_from_file("test_data.csv",
                      correct_types = np.repeat([int], [len(data_test.columns)]),
                                response_header = 'Y',
                               train_percentage = 0.0)
BBA_data = (headers, train_BBA, test_BBA, response_header, features_to_ignore, correct_types)

In [39]:
class HirePredictorBuilder(AbstractModelFactory):
    def __init__(self, *args, **kwargs):
        AbstractModelFactory.__init__(self, *args, **kwargs)
        self.verbose_factory_name = "HirePredictor"
    def build(self, train_set):
        return HirePredictor()

class HirePredictor(AbstractModelVisitor):
    def __init__(self):
        with open( 'clf.pickle', 'rb' ) as f:
            self.clf = pickle.load(f)

    def test(self, test_set, test_name=""):
        df_test = pd.DataFrame(
            test_set, columns =data_test.columns.to_list())
        targets = df_test['Y']
        preds = self.clf.predict(df_test.drop('Y', axis=1))
        return [[a,b] for (a,b) in zip(targets, preds)]

In [40]:
auditor = BBA.Auditor()
auditor.ModelFactory = HirePredictorBuilder
auditor(BBA_data, output_dir = "audit-output")

Training initial model. (17:39:47)
Calculating original model statistics on test data:
	Training Set:
		Conf-Matrix: {0: {0: 58609, 1: 230}, 1: {1: 3365}}
		accuracy: 0.9963024885859431
		BCR: 0.9980455140298101
	Testing Set:
		Conf-Matrix {0: {0: 26016, 1: 2701}, 1: {0: 1009, 1: 684}}
		accuracy: 0.8780006576783953
		BCR: 0.654980376458612
Auditing: 'trajet' (1/24). (17:39:50)
Auditing: 'catr' (2/24). (17:40:18)
Auditing: 'circ' (3/24). (17:40:48)
Auditing: 'nbv' (4/24). (17:41:19)
Auditing: 'prof' (5/24). (17:41:50)
Auditing: 'plan' (6/24). (17:42:23)
Auditing: 'surf' (7/24). (17:42:57)
Auditing: 'infra' (8/24). (17:43:31)
Auditing: 'situ' (9/24). (17:44:05)
Auditing: 'vma' (10/24). (17:44:40)
Auditing: 'mois' (11/24). (17:45:10)
Auditing: 'lum' (12/24). (17:45:36)
Auditing: 'dep' (13/24). (17:46:07)
Auditing: 'agg' (14/24). (17:46:32)
Auditing: 'int' (15/24). (17:47:04)
Auditing: 'atm' (16/24). (17:47:36)
Auditing: 'col' (17/24). (17:48:11)
Auditing: 'catv' (18/24). (17:48:39)
Audit

## Interprétation du modèle

### Interprétation avec ShapKit

Création d'une fonction dans `utils` qui va paralléliser les calculs des valeurs de shapley.
On va regarder les contributions dans le cas où les prédictions sont 1 et dans les cas où les prédictions sont 0 

In [41]:
# t prend les entrées qui donne une sortie 1
# t2 prend le reste
t = []
t2 = []
for i in range(0, len(preds)):
    if preds[i] == 1:
        t.append(i)
    else:
        t2.append(i)

In [ ]:
# Calcul de shap en parallèle pour 10 valeurs
# Ici pour les sorties données 1
s = parallel_shap(data_test, X_train, clf, t, n_iter=10)
s2 = parallel_shap(data_test, X_train, clf, t2, n_iter=10)

In [ ]:
s.start_shap()
s2.start_shap()

In [ ]:
# Si intérruption du processus
s.stop_shap()

In [ ]:
from shapkit.plots import plot_shapley

# Affichage des résultats
for i in range(0, 10):
    true_shap, query_instance, reference = s.results[i]
    fig = plot_shapley(x=query_instance, fc=s.fc, ref=reference, shapley_values=true_shap, n_attributes=24)

In [ ]:
from shapkit.plots import plot_shapley

# Affichage des résultats
for i in range(0, 10):
    true_shap, query_instance, reference = s2.results[i]
    fig = plot_shapley(x=query_instance, fc=s2.fc, ref=reference, shapley_values=true_shap, n_attributes=24)

## Mitigation des données via pre-traitements

### I- Managment des Données avec AIF360

### II- 